Connected to venv (Python 3.11.5)

In [102]:
import os

In [103]:
train_path = "./rock_paper_scissors/train/"
val_path   = "./rock_paper_scissors/val/"

In [104]:
def getFileNames(path):
    # Get all the file names from the path that end with .jpg
    file_names = [f for f in os.listdir(path) if f.endswith(".jpg")]
    return file_names

def getLabels(path):
    labels = [f for f in os.listdir(path) if f.endswith(".txt")]
    return labels

In [105]:
training_file_names = getFileNames(train_path)
training_file_labels = getLabels(train_path)
validation_file_names = getFileNames(val_path)
validation_file_labels = getLabels(val_path)

In [106]:
types = ["rock", "paper", "scissors"]

In [185]:
def readLabelFile(path):
    with open(path) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    split = content[0].split(" ")
    t = int(split[0])
    
    x = float(split[1])
    y = float(split[2])
    w = float(split[3])
    h = float(split[4])
    
    if t == 0:
        return np.array([1, 0, 0, x, y, w, h])
    elif t == 1:
        return np.array([0, 1, 0, x, y, w, h])
    elif t == 2:
        return np.array([0, 0, 1, x, y, w, h])
    
    

In [186]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

In [187]:
images = np.array([plt.imread(train_path + training_file_names[0])])
images = images / 255


In [188]:
labels = np.array([readLabelFile(train_path + training_file_labels[0])])

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(480, 640, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(7, activation='sigmoid'),
])


In [ ]:
model.compile(optimizer="adam", loss="mse")

In [ ]:
images[0].shape

(480, 640, 3)

In [ ]:
# add a dimension to the labels
print(images, labels)

In [196]:
model.fit(images, labels, epochs=100)

Epoch 1/100


1/1 [==============================] - 0s 186ms/step - loss: 0.0014
Epoch 2/100
1/1 [==============================] - 0s 184ms/step - loss: 9.0668e-04
Epoch 3/100
1/1 [==============================] - 0s 167ms/step - loss: 4.6176e-04
Epoch 4/100
1/1 [==============================] - 0s 174ms/step - loss: 8.3580e-04
Epoch 5/100
1/1 [==============================] - 0s 181ms/step - loss: 6.8398e-04
Epoch 6/100
1/1 [==============================] - 0s 181ms/step - loss: 2.0619e-04
Epoch 7/100
1/1 [==============================] - 0s 168ms/step - loss: 4.2415e-04
Epoch 8/100
1/1 [==============================] - 0s 163ms/step - loss: 5.5778e-04
Epoch 9/100
1/1 [==============================] - 0s 167ms/step - loss: 2.5578e-04
Epoch 10/100
1/1 [==============================] - 0s 170ms/step - loss: 3.4391e-04
Epoch 11/100
1/1 [==============================] - 0s 172ms/step - loss: 5.8647e-04
Epoch 12/100
1/1 [==============================] - 0s 169ms/step - loss: 3.8694e-04
Epoch

In [197]:
model.predict(images)

1/1 [==============================] - 0s 57ms/step


array([[1.3171020e-04, 9.9949855e-01, 6.2797684e-05, 4.8667970e-01,
        6.2595475e-01, 3.2981879e-01, 3.2719821e-01]], dtype=float32)

In [198]:
# predict on validation set
validation_images = np.array([plt.imread(val_path + validation_file_names[0])])
validation_images = validation_images / 255
validation_labels = np.array([readLabelFile(val_path + validation_file_labels[0])])
model.predict(validation_images)


1/1 [==============================] - 0s 57ms/step


array([[-0.03798455,  0.87627155,  0.03496788,  0.33867007,  0.4784958 ,
         0.24757263,  0.24404348]], dtype=float32)

In [203]:
# get the predicted bounding box
prediction = model.predict(validation_images)
prediction = prediction[0]
prediction

1/1 [==============================] - 0s 42ms/step


array([-0.03798455,  0.87627155,  0.03496788,  0.33867007,  0.4784958 ,
        0.24757263,  0.24404348], dtype=float32)

In [204]:
# draw the predicted bounding box
from PIL import ImageDraw

img = Image.open(val_path + validation_file_names[0])

draw = ImageDraw.Draw(img)
draw.rectangle([(prediction[3], prediction[4]), (prediction[3] + prediction[5], prediction[4] + prediction[6])], outline="red")
img.show()

In [ ]:
# draw the actual bounding box on webcam
